In [1]:
import rx
import rxsci as rs

from IPython.display import Image

In [2]:
source = [1, 2, 3, 4, 5, 6, 7, 8]

# Stateless Processing

In [3]:
rx.from_(source).pipe(
    rs.ops.map(lambda i: i*2)
).subscribe(on_next=print)

2
4
6
8
10
12
14
16


In [4]:
rx.from_(source).pipe(
    rs.ops.filter(lambda i: i%2 == 0)
).subscribe(on_next=print)

2
4
6
8


# Stateful Processing

In [4]:
rx.from_(source).pipe(
    rs.state.with_memory_store(rx.pipe(
        rs.math.sum(reduce=True)
    )),
).subscribe(on_next=print)

36.0


## windowing

In [6]:
Image(url='./marble-roll-3.png')

In [22]:
rx.from_(source).pipe(
    rs.state.with_memory_store(rx.pipe(
        rs.data.roll(window=3, stride=3, pipeline=rx.pipe(
            rs.math.mean(reduce=True),
        )),
    )),
).subscribe(on_next=print)

2.0
5.0
7.5


## grouping

In [8]:
Image(url='./marble-group-by.png')

In [9]:
rx.from_(source).pipe(
    rs.state.with_memory_store(rx.pipe(
        rs.ops.group_by(lambda i: i%2, pipeline=rx.pipe(
            rs.math.sum(reduce=True),
        )),
    )),
).subscribe(on_next=print)

16.0
20.0


# Branching

In [7]:
def my_op():
    return rx.pipe(
        rs.ops.tee_map(
            rs.math.sum(reduce=True),
            rs.math.mean(reduce=True),
        )
    )
    

rx.from_(source).pipe(
    my_op(),
).subscribe(on_next=print)

(36.0, 4.5)


# Composition

In [11]:
rx.from_(source).pipe(
    rs.state.with_memory_store(rx.pipe(
        rs.ops.group_by(lambda i: i%2, pipeline=rx.pipe(
            rs.data.roll(window=2, stride=2, pipeline=rx.pipe(
                rs.math.sum(reduce=True),
            )),
        )),
    )),
).subscribe(on_next=print)

4.0
6.0
12.0
14.0
